# 1. Menghitung HAZ-Score

## Import Library

In [66]:
import gdown
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Understanding

In [137]:
dataset = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset.csv')
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,Laki-laki,19,91.6,13.3
1,1,Laki-laki,20,77.7,8.5
2,2,Laki-laki,10,79.0,10.3
3,3,Perempuan,2,50.3,8.3
4,4,Perempuan,5,56.4,10.9


## Data Preparation

### Encoding Data Kategori

In [138]:
label = LabelEncoder()
dataset['Jenis Kelamin'] = label.fit_transform(dataset['Jenis Kelamin'])
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,0,19,91.6,13.3
1,1,0,20,77.7,8.5
2,2,0,10,79.0,10.3
3,3,1,2,50.3,8.3
4,4,1,5,56.4,10.9


### Hitung HAZ-Score

In [139]:
# who_growth_standards = {
#     "Laki-laki": {
#         0: {"median": 49.9, "sd": 3.1},
#         6: {"median": 67.6, "sd": 2.6},
#         12: {"median": 76.1, "sd": 2.9},
#         24: {"median": 87.1, "sd": 3.1},
#         36: {"median": 95.2, "sd": 3.5},
#         48: {"median": 102.3, "sd": 3.8},
#         60: {"median": 109.2, "sd": 4.2},
#     },
#     "Perempuan": {
#         0: {"median": 49.1, "sd": 1.8},
#         6: {"median": 65.7, "sd": 2.5},
#         12: {"median": 74.0, "sd": 2.8},
#         24: {"median": 85.7, "sd": 3.1},
#         36: {"median": 94.0, "sd": 3.5},
#         48: {"median": 101.6, "sd": 3.8},
#         60: {"median": 108.5, "sd": 4.2},
#     },
# }

# def calculate_haz(age_months, height_cm, gender):
#     if gender not in [0, 1]:
#         return None, None

#     gender_key = "Laki-laki" if gender == 0 else "Perempuan"

#     # Handle NaN in age or height
#     if pd.isna(age_months) or pd.isna(height_cm):
#         return None, None

#     # Temukan usia referensi terdekat
#     available_ages = who_growth_standards[gender_key].keys()
#     if not available_ages:
#         return None, None

#     nearest_age = min(available_ages, key=lambda x: abs(x - age_months))
#     ref = who_growth_standards[gender_key][nearest_age]

#     # Handle potential division by zero if sd is 0, although unlikely with growth standards
#     if ref["sd"] == 0:
#          return None, "error: zero standard deviation"

#     haz = (height_cm - ref["median"]) / ref["sd"]

#     # Klasifikasi WHO - 3 kelas
#     if haz < -2:
#         status = "Stunting"
#     elif haz < -1.5:
#         status = "Berpotensi stunting"
#     else:
#         status = "Normal"

#     return round(haz, 2), status

# haz_results = dataset.apply(
#     lambda row: calculate_haz(row["Umur (bulan)"], row["Tinggi Badan (cm)"], row["Jenis Kelamin"]), axis=1
# )

# haz_results_df = pd.DataFrame(haz_results.tolist(), columns=["HAZ", "Status"])
# dataset = pd.concat([dataset, haz_results_df], axis=1)




In [140]:
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,0,19,91.6,13.3
1,1,0,20,77.7,8.5
2,2,0,10,79.0,10.3
3,3,1,2,50.3,8.3
4,4,1,5,56.4,10.9


In [141]:
# # Contoh WHO Height-for-Age Z-score lookup
# Contoh WHO Height-for-Age Z-score lookup
who_haz_lookup = {
    (0, 1): (49.9, 1.9),
    (0, 0): (49.1, 1.8),
    (6, 1): (67.6, 2.6),
    (6, 0): (65.7, 2.5),
    (12, 1): (76.1, 2.9),
    (12, 0): (74.0, 2.8),
    (24, 1): (87.1, 3.1),
    (24, 0): (85.7, 3.0),
    (36, 1): (95.2, 3.5),
    (36, 0): (94.0, 3.4),
    (48, 1): (102.3, 3.8),
    (48, 0): (101.6, 3.7),
    (60, 1): (109.2, 4.2),
    (60, 0): (108.5, 4.2),
}

def calculate_haz(row):
    umur = row["Umur (bulan)"]
    jk = row["Jenis Kelamin"]
    tinggi = row["Tinggi Badan (cm)"]

    # Find the closest age group for the given gender
    closest_key = None
    min_diff = float('inf')
    for (lookup_age, lookup_jk) in who_haz_lookup.keys():
        if lookup_jk == jk:
            diff = abs(lookup_age - umur)
            if diff < min_diff:
                min_diff = diff
                closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age < umur: # Prefer the closest age less than current if tie
                 closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age > umur and closest_key and closest_key[0] > umur: # If closest so far is also > umur, take this one if closer
                 closest_key = (lookup_age, lookup_jk)


    if closest_key in who_haz_lookup:
        median, sd = who_haz_lookup[closest_key]
        # Handle potential division by zero if SD is 0
        if sd != 0:
            return round((tinggi - median) / sd, 2)
        else:
            return None # Return None if SD is 0 to avoid error
    return None

dataset["HAZ"] = dataset.apply(calculate_haz, axis=1)

# Klasifikasi stunting
def classify_stunting(haz):
    if pd.isna(haz):
        return "Unknown"
    elif haz < -2:
        return "Stunting"
    elif haz < -1.5:
        return "Berpotensi Stunting"
    else:
        return "Normal"


dataset["Stunting_Status"] = dataset["HAZ"].apply(classify_stunting)
dataset.to_csv("/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset_with_status.csv", index=False)

dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.97,Normal
1,1,0,20,77.7,8.5,-2.67,Stunting
2,2,0,10,79.0,10.3,1.79,Normal
3,3,1,2,50.3,8.3,0.21,Normal
4,4,1,5,56.4,10.9,-4.31,Stunting


# 2. Klasifikasi Stunting

## Data Understanding

In [142]:
stunting = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset_with_status.csv')

# Display basic info and first few rows
stunting.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.97,Normal
1,1,0,20,77.7,8.5,-2.67,Stunting
2,2,0,10,79.0,10.3,1.79,Normal
3,3,1,2,50.3,8.3,0.21,Normal
4,4,1,5,56.4,10.9,-4.31,Stunting


In [143]:
stunting['Stunting_Status'].value_counts()

,count
Stunting_Status,
Normal,78938
Stunting,15298
Berpotensi Stunting,5764


In [144]:
stunting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Id                 100000 non-null  int64  
 1   Jenis Kelamin      100000 non-null  int64  
 2   Umur (bulan)       100000 non-null  int64  
 3   Tinggi Badan (cm)  100000 non-null  float64
 4   Berat Badan (kg)   100000 non-null  float64
 5   HAZ                100000 non-null  float64
 6   Stunting_Status    100000 non-null  object 
dtypes: float64(3), int64(3), object(1)
memory usage: 5.3+ MB


## Encoding Data

In [145]:
label = LabelEncoder()
stunting['Stunting_Status'] = label.fit_transform(stunting['Stunting_Status'])
stunting.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.97,1
1,1,0,20,77.7,8.5,-2.67,2
2,2,0,10,79.0,10.3,1.79,1
3,3,1,2,50.3,8.3,0.21,1
4,4,1,5,56.4,10.9,-4.31,2


## Feature *Scaling*

In [146]:
fitur = ['Jenis Kelamin', 'Umur (bulan)', 'Tinggi Badan (cm)', 'Berat Badan (kg)']

scaler = StandardScaler()
stunting[fitur] = scaler.fit_transform(stunting[fitur])

stunting.head()


,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,-0.996426,0.973302,1.625534,1.224185,1.97,1
1,1,-0.996426,1.112198,0.402027,-0.230024,-2.67,2
2,2,-0.996426,-0.276761,0.516456,0.315304,1.79,1
3,3,1.003586,-1.387929,-2.009777,-0.290616,0.21,1
4,4,1.003586,-0.971241,-1.472843,0.497080,-4.31,2


## Data Splitting

In [147]:
# 5. Pilih fitur dan target
X = stunting[["Jenis Kelamin", "Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)"]]
y = stunting["Stunting_Status"]

# 6. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelling

In [148]:
# 8. Model klasifikasi
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 9. Evaluasi model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 10. Membuat DataFrame hasil prediksi
df_prediksi = X_test.copy()
df_prediksi["Actual_Label"] = label.inverse_transform(y_test)
df_prediksi["Predicted_Label"] = label.inverse_transform(y_pred)

# 11. Tampilkan 10 hasil prediksi pertama
df_prediksi.head(10)

Classification Report:
                      precision    recall  f1-score   support

Berpotensi Stunting       1.00      1.00      1.00      1186
             Normal       1.00      1.00      1.00     15764
           Stunting       1.00      1.00      1.00      3050

           accuracy                           1.00     20000
          macro avg       1.00      1.00      1.00     20000
       weighted avg       1.00      1.00      1.00     20000

Confusion Matrix:
 [[ 1186     0     0]
 [    0 15764     0]
 [    0     0  3050]]


,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),Actual_Label,Predicted_Label
75721,1.003586,-1.526825,-1.631283,-0.684465,Normal,Normal
80184,1.003586,-0.554553,-0.627831,0.890928,Normal,Normal
19864,-0.996426,1.112198,1.484698,0.497080,Normal,Normal
76699,-0.996426,0.139927,0.815731,0.103232,Normal,Normal
92991,-0.996426,-0.137865,-0.266941,1.193889,Normal,Normal
76434,-0.996426,0.556614,0.604478,0.739448,Normal,Normal
84004,1.003586,-0.137865,-0.284545,0.769744,Stunting,Stunting
80917,-0.996426,-0.832345,-0.619029,-0.805649,Normal,Normal
60767,-0.996426,-1.110137,-1.173568,-0.139136,Stunting,Stunting
50074,1.003586,0.834406,0.085148,1.527145,Normal,Normal


In [77]:
# Model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


callbacks = [
    EarlyStopping(
        monitor='val_loss',        # Pantau nilai loss di data validasi
        patience=5,                # Berhenti training jika tidak membaik selama 5 epoch
        restore_best_weights=True # Kembalikan bobot terbaik (bukan terakhir)
    ),
    ModelCheckpoint(
        "/content/drive/MyDrive/Capstone-ML/Capstone/best_model.h5",
        monitor='val_accuracy',   # Simpan model terbaik berdasarkan akurasi validasi
        save_best_only=True       # Simpan hanya jika model membaik
    )
]


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.1,
    callbacks=callbacks
)

# Evaluation
model.evaluate(X_test, y_test)
model.save("/content/drive/MyDrive/Capstone-ML/Capstone/model_stunting.h5")

Epoch 1/20
4481/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7971 - loss: 0.5293

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.7973 - loss: 0.5288 - val_accuracy: 0.8709 - val_loss: 0.3340
Epoch 2/20
4481/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8716 - loss: 0.3352

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8716 - loss: 0.3352 - val_accuracy: 0.8856 - val_loss: 0.3000
Epoch 3/20
4493/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8830 - loss: 0.3081

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.8830 - loss: 0.3081 - val_accuracy: 0.8938 - val_loss: 0.2746
Epoch 4/20
4492/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8905 - loss: 0.2826

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.8905 - loss: 0.2825 - val_accuracy: 0.9044 - val_loss: 0.2507
Epoch 5/20
4489/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9010 - loss: 0.2570

...

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9541 - loss: 0.1150 - val_accuracy: 0.9536 - val_loss: 0.1095
Epoch 16/20
4493/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9590 - loss: 0.1075

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9590 - loss: 0.1075 - val_accuracy: 0.9639 - val_loss: 0.0954
Epoch 17/20
4498/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9611 - loss: 0.1026

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9611 - loss: 0.1026 - val_accuracy: 0.9657 - val_loss: 0.0904
Epoch 18/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9645 - loss: 0.0980 - val_accuracy: 0.9621 - val_loss: 0.0974
Epoch 19/20
4480/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9671 - loss: 0.0926

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9671 - loss: 0.0926 - val_accuracy: 0.9674 - val_loss: 0.0841
Epoch 20/20
4480/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9688 - loss: 0.0855

4500/4500 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9688 - loss: 0.0855 - val_accuracy: 0.9709 - val_loss: 0.0810
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9704 - loss: 0.0865


In [78]:
def predict_stunting(model, scaler, input_data):
    """
    input_data: dict seperti
        {
            "Umur (bulan)": 24,
            "Tinggi Badan (cm)": 85.0,
            "Berat Badan (kg)": 12.5,
            "JK_Code": 1
        }
    """
    df_input = pd.DataFrame([input_data])
    scaled_input = scaler.transform(df_input)
    pred_probs = model.predict(scaled_input)
    pred_class = tf.argmax(pred_probs, axis=1).numpy()[0]
    return label.inverse_transform([pred_class])[0]

In [79]:
save_path = '/content/Model'
tf.saved_model.save(model, save_path)

In [80]:
!pip install tensorflowjs

In [81]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  '/content/Model'\
  '/content/drive/MyDrive/Capstone-ML/Capstone/Model-tfjs'

2025-05-23 08:52:50.517344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747990370.544692   10871 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747990370.552618   10871 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-05-23 08:53:03.583862: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1747990383.841419   10871 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00

## Inference Model

In [95]:
# Load model dari file .h5
model = load_model('/content/drive/MyDrive/Capstone-ML/Capstone/best_model.h5')

# Cek ringkasan arsitektur
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 245 (984.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [96]:
predict_data = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan.csv')
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01


In [97]:
predict = predict_data.copy()
predict.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01


In [98]:
label = LabelEncoder()
predict['Gender'] = label.fit_transform(predict['Gender'])
predict.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,0,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,0,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,0,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,0,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,0,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,1,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,1,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,1,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,0,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,1,32,10.7,79.0,2022-02-01


In [99]:
scaler = StandardScaler()
fitur = ['Gender','Age','Height','Weight']
predict[fitur] = scaler.fit_transform(predict[fitur])
predict.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,-1.105407,0.299261,-0.291210,-0.298971,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,-1.105407,-0.750513,-0.377340,-0.719280,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,-1.105407,-1.137272,-0.664439,-0.649229,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,-1.105407,0.244010,0.340408,0.751802,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,-1.105407,-0.198000,-0.262501,0.191390,2022-02-01


In [100]:
import numpy as np
X = predict[fitur].values
prediksi = model.predict(X)

predicted_class = np.argmax(prediksi, axis=1)

predict['prediction'] = predicted_class
predict_data['Stunting_Status'] = predict['prediction'].copy()

predict_data.head(10)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date,Stunting_Status
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01,0
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01,1
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01,1
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01,1
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01,1
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01,1
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01,1
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01,1
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01,1
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01,2


In [101]:
label_map = {
    1: 'Normal',
    2: 'Stunting',
    0: 'Berpotensi Stunting'
}

predict_data['Stunting_Status'] = [label_map[i] for i in predict_data['Stunting_Status']]

# Simpan ke
predict_data.to_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan_with_status.csv', index=False)

In [102]:
predict_data['Stunting_Status'].unique()

array(['Berpotensi Stunting', 'Normal', 'Stunting'], dtype=object)

In [104]:
predict_data['Stunting_Status'].value_counts()

,count
Stunting_Status,
Normal,742
Stunting,51
Berpotensi Stunting,38


In [103]:
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date,Stunting_Status
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01,Berpotensi Stunting
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01,Normal
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01,Normal
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01,Normal
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01,Normal
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01,Normal
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01,Normal
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01,Normal
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01,Normal
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01,Stunting
